先重整流程，然后再想办法完善数据，比如东财

## import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
import QUANTAXIS as QA
import talib as ta
import datetime, time

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## import & update 60-min data

In [3]:
# read non-update data
data = pd.read_excel('index_60min.xlsx',index_col=0, header=0)

In [7]:
# update data
# set update data date
start_update = QA.QA_util_get_next_day(data.index[-1].strftime('%Y-%m-%d'), 1)
end_update = QA.QA_util_get_last_day(datetime.datetime.today().strftime('%Y-%m-%d'), 1)

# retrieve & adjust cols for concating
data_update = QA.QA_fetch_get_index_min('tdx', '000001', start_update, end_update, '60min')
data_update = data_update.loc[:,['open', 'close', 'high', 'low', 'vol', 'amount', 'up_count',
       'down_count', 'code', 'date', 'date_stamp', 'time_stamp', 'type']]

# concat data
data = pd.concat([data, data_update], axis=0)

# save data
writer = pd.ExcelWriter('index_60min.xlsx')
data.to_excel(writer)
writer.save()

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 60.191.117.167 future_ip 202.103.36.71


In [4]:
# resample data
data_min = data[(data.index.hour == 10) | (data.index.hour == 11)].loc[:, 
    ['open', 'high', 'low', 'close', 'vol', 'amount']]
data_min_resample_day = data_min.resample('D').apply({
    'open':'first', 'high':'max', 'low':'min', 'close':'last',
    'vol':'sum','amount':'sum'
}).dropna()

In [5]:
end_update = QA.QA_util_get_last_day(datetime.datetime.today().strftime('%Y-%m-%d'), 1)
start = '2016-11-16'
end = end_update
index_day = QA.QA_fetch_get_index_day('tdx', '000001', start, end)

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 60.191.117.167 future_ip 202.103.36.71


test method accuracy

In [6]:
data_half_day = data_min_resample_day
data_full_day = index_day

In [7]:
half_day_change = (data_half_day.close / data_full_day.close.shift(1))[1:] - 1
full_day_change = data_full_day.close.pct_change()[1:]

In [25]:
ind = half_day_change >= 0.01
half_filter = half_day_change[ind]
full_filter = full_day_change[ind]
result = (np.sign(half_filter) * np.sign(full_filter)).value_counts()

In [26]:
result.loc[1.0] / result.sum()

1.0

In [28]:
half_filter

datetime
2017-04-05    0.010600
2017-08-25    0.011554
2017-10-09    0.012502
2018-01-02    0.010828
2018-02-13    0.018018
2018-02-22    0.019502
2018-04-24    0.020701
2018-05-07    0.012070
2018-05-31    0.013826
2018-06-29    0.011995
2018-07-09    0.016504
2018-07-12    0.018961
2018-07-24    0.015873
2018-08-07    0.014251
2018-08-09    0.017846
2018-08-21    0.014327
2018-08-27    0.014278
2018-09-26    0.012689
2018-10-22    0.041718
2018-10-24    0.015288
2018-10-31    0.011293
2018-11-01    0.011311
2018-11-02    0.012052
2018-12-03    0.029078
2018-12-13    0.016006
Name: close, dtype: float64

In [33]:
full_filter

date
2017-04-05    0.014833
2017-08-25    0.018343
2017-10-09    0.007596
2018-01-02    0.012446
2018-02-13    0.009774
2018-02-22    0.021693
2018-04-24    0.019857
2018-05-07    0.014756
2018-05-31    0.017765
2018-06-29    0.021716
2018-07-09    0.024709
2018-07-12    0.021560
2018-07-24    0.016093
2018-08-07    0.027433
2018-08-09    0.018334
2018-08-21    0.013104
2018-08-27    0.018857
2018-09-26    0.009230
2018-10-22    0.040938
2018-10-24    0.003264
2018-10-31    0.013524
2018-11-01    0.001329
2018-11-02    0.026951
2018-12-03    0.025736
2018-12-13    0.012259
Name: close, dtype: float64

In [39]:
data_full_day.close[-50:-40:]

date
2018-10-23    2594.83
2018-10-24    2603.30
2018-10-25    2603.80
2018-10-26    2598.85
2018-10-29    2542.10
2018-10-30    2568.05
2018-10-31    2602.78
2018-11-01    2606.24
2018-11-02    2676.48
2018-11-05    2665.43
Name: close, dtype: float64

In [32]:
index_day[1:][ind]

,open,close,high,low,vol,amount,up_count,down_count,date,code,date_stamp
date,,,,,,,,,,,
2017-04-05,3235.66,3270.31,3270.65,3233.24,2483202.0,2.732012e+11,1123,89,2017-04-05,000001,1.491322e+09
2017-08-25,3271.46,3331.52,3331.91,3271.46,2058394.0,2.271410e+11,1170,112,2017-08-25,000001,1.503590e+09
2017-10-09,3403.25,3374.38,3410.17,3366.97,1917360.0,2.274406e+11,1080,224,2017-10-09,000001,1.507478e+09
2018-01-02,3314.03,3348.33,3349.05,3314.03,2022788.0,2.277885e+11,1108,230,2018-01-02,000001,1.514822e+09
2018-02-13,3176.11,3184.96,3219.22,3176.11,1515276.0,1.720452e+11,741,550,2018-02-13,000001,1.518451e+09
2018-02-22,3237.57,3268.56,3269.92,3234.12,1387304.0,1.573039e+11,1279,54,2018-02-22,000001,1.519229e+09
2018-04-24,3069.75,3128.93,3136.04,3069.75,1622748.0,1.955450e+11,1301,55,2018-04-24,000001,1.524499e+09
2018-05-07,3094.90,3136.64,3136.84,3091.66,1389481.0,1.782439e+11,1274,78,2018-05-07,000001,1.525622e+09
2018-05-31,3061.83,3095.47,3098.08,3054.27,1404755.0,1.851990e+11,1209,152,2018-05-31,000001,1.527696e+09


In [3]:
def update_data(data, start, end):
    """
    Implements:
        update the 60mins data & save the data to the excel file
    
    Arguments:
        data -- dataframe type, 60mins data that to be updating
        start -- str, the data updated start date
        end -- str, the data updated end date
    
    Returns:
        data
    """
    # retrieve & adjust cols for concating
    data_update = QA.QA_fetch_get_index_min('tdx', '000001', start, end, '60min')
    data_update = data_update.loc[:,['open', 'close', 'high', 'low', 'vol', 'amount', 'up_count',
           'down_count', 'code', 'date', 'date_stamp', 'time_stamp', 'type']]

    # concat data
    data = pd.concat([data, data_update], axis=0)

    # save data
    writer = pd.ExcelWriter('index_60min.xlsx')
    data.to_excel(writer)
    writer.save()
    
    return data

In [4]:
def update_and_import_data():
    """
    Implements:
        update the 60min data
        import the data
    
    Arguments:
        --
    
    Returns:
        data_min_resample_day
        index_day
    """
    # read non-update data
    data = pd.read_excel('index_60min.xlsx',index_col=0, header=0)
    
    start_update = QA.QA_util_get_next_day(data.index[-1].strftime('%Y-%m-%d'), 1)
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    end_update = QA.QA_util_get_last_day(today, 1)
    
    if start_update < today:
        data = update_data(data, start_update, end_update)
    
    # resample data
    data_min = data[(data.index.hour == 10) | (data.index.hour == 11)].loc[:, 
        ['open', 'high', 'low', 'close', 'vol', 'amount']]
    data_min_resample_day = data_min.resample('D').apply({
        'open':'first', 'high':'max', 'low':'min', 'close':'last',
        'vol':'sum','amount':'sum'
    }).dropna()
    
    start = '2016-11-16'
    end = end_update
    index_day = QA.QA_fetch_get_index_day('tdx', '000001', start, end)
    
    return (data_min_resample_day, index_day)

In [9]:
def calc_right_nums(data_half_day, data_full_day, cur_half_day_range):
    """
    Implements:
        依据半天的涨幅，计算出有多少半天的涨幅和全天涨幅一致的数量和整体数量
    Arguments:
        
    Returns:
        
    """
    # 计算半天的涨幅和全天的涨幅
    half_day_change = (data_half_day.close / data_full_day.close.shift(1))[1:] - 1
    full_day_change = data_full_day.close.pct_change()[1:]
    
    if cur_half_day_range >= 0:
        ind = half_day_change >= cur_half_day_range
    else:
        ind = half_day_change <= cur_half_day_range
    
    half_filter = half_day_change[ind]
    full_filter = full_day_change[ind]
    
    result = (np.sign(half_filter) * np.sign(full_filter)).value_counts()
    right_nums = result.loc[1.0]
    total_nums = result.sum()
    
    return (right_nums, total_nums)

In [10]:
def Kelly_Criterion(p, q, bet_win, bet_loss):
    """
    Implements:
        依据当前的半日涨跌幅, 赔率计算仓位
    Arguments:
        
    Returns:
        
    """
    f_star = p - q / bet_win
    f_star_reverse = q - p / bet_loss
    
    return (f_star, f_star_reverse)

In [24]:
def calc_probability_and_position(data_half_day, data_full_day, cur_half_day_range, bet_win, bet_loss):
    """
    Implements:
        
    Arugments:
        
    Returns:
        
    """
    right_nums, total_nums = calc_right_nums(data_half_day, data_full_day, cur_half_day_range)
    if cur_half_day_range >= 0:
        p = right_nums / total_nums
        q = 1 - p
    else:
        q = right_nums / total_nums
        p = 1 - q
        
    f_star, f_star_reverse = Kelly_Criterion(p, q, bet_win, bet_loss)
    return (f_star, f_star_reverse)

In [5]:
data_half_day, data_full_day = update_and_import_data()

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 60.191.117.167 future_ip 202.103.36.71


In [34]:
half_range = 0 * 1e-4
bet_win = 0.47
bet_loss = 1.72

In [35]:
f_star, f_star_reverse = calc_probability_and_position(data_half_day, data_full_day,
                                                       half_range, bet_win, bet_loss)
print(f_star, f_star_reverse)

0.40863579474342926 -0.2823920265780731


In [20]:
int(f_star * 12000)

33